In [ ]:
import numpy as np
def convert_fen_to_nparray(file_string):
  fen_to_int = dict()
  fen_to_int["r"] = 1
  fen_to_int["n"] = 2
  fen_to_int["b"] = 3
  fen_to_int["q"] = 4
  fen_to_int["k"] = 5
  fen_to_int["p"] = 6
  fen_to_int["R"] = 7
  fen_to_int["N"] = 8
  fen_to_int["B"] = 9
  fen_to_int["Q"] = 10
  fen_to_int["K"] = 11
  fen_to_int["P"] = 12
  board = np.zeros((8,8),dtype="float32")
  row = 0
  column = 0
  for i in file_string:
    try:
      i = int(i)
      for k in range(i):
        column += 1
    except:
      if i == "-":
        row += 1
        column = 0
      else:
        board[row,column]=fen_to_int[i]
        column += 1
  return board


def to_one_hot(labels):
  results = np.zeros((len(labels)*8,13),dtype="float32")
  count = 0
  for row in labels:
    for element in row:
      results[count,int(element)] = 1
      count += 1
  return results




In [ ]:
def image_preprocessor(filename):
  import PIL
  from IPython.display import Image, display
  from matplotlib import pyplot as plt
  from PIL import Image as im
  from PIL import Resampling
  img = PIL.Image.open(filename)
  img = img.resize((400,400),resample=PIL.Image.Resampling.BOX)

  left = 0
  top = 0
  right = 50
  bottom = 50
  count = 0

  train_data =np.zeros((64,50,50,3),dtype="float32")

  for i in range(64):
    #(left,top,right,bottom)
    img = img.crop([left,top,right,bottom])
    left += 50
    right += 50
    count += 1
    if count% 8 == 0:
      bottom += 50
      top += 50
      left = 0
      right = 50
    img.save(f"{str(count)}.png")
    train_data[count-1] = np.asarray(img)
    display(Image(f"{str(count)}.png"))
    img = PIL.Image.open(filename)
    img = img.resize((400,400),resample=PIL.Image.Resampling.BOX)
  #greyscale
  train_data = np.dot(train_data[...,:3],[0.2989, 0.5870, 0.1140])
  #normalize
  train_data /= 255
  return train_data


In [ ]:
import os
from IPython.display import Image, display
from PIL import Image as im
master_x_test = np.empty((64000,50,50,3))
master_y_test = np.empty((0,8))
file_count = 0
count = 0
for filename in os.listdir("/content/test"):
   if file_count == 1000:
     break
   else:
     with open(os.path.join("/content/test", filename), 'r') as f:
       current_x_train = image_preprocessor(f"/content/test/{filename}")
       for square in current_x_train:
         master_x_test[count] = square
         count+=1
       current_y_train = convert_fen_to_nparray(filename[:-5])
       master_y_test = np.vstack((master_y_test,current_y_train))
       file_count+=1
       if file_count%100==0:
         print(file_count)

master_x_test = np.dot(master_x_test[...,:3],[0.2989, 0.5870, 0.1140])
print(master_x_test.shape)
master_x_test /= 255
print(master_x_test.shape)
master_y_test = to_one_hot(master_y_test)
print(master_y_test.shape)
# print(master_x_train[0])
# print(master_y_train[0])

100
200
300
400
500
600
700
800
900
1000
(64000, 50, 50)
(64000, 50, 50)
(64000, 13)


In [ ]:
np.save("x_test1000",master_x_test)
np.save("y_test1000",master_y_test)

In [ ]:
master_x_train=np.load("/content/x_train1000.npy")
master_y_train=np.load("/content/y_train1000.npy")
print(master_x_train.shape)
print(master_y_train.shape)


(64000, 50, 50)
(64000, 13)


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile

model = Sequential()
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', input_shape = (50, 50, 1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(units = 128, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dense(units = 13, activation = 'softmax'))
model.summary()
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'],)
history = model.fit(x = master_x_train, y= master_y_train,
                    epochs = 10, batch_size = 64,
                    verbose = 1,
                    validation_split = 0.25,
                    callbacks = [EarlyStopping(min_delta = .001, patience = 5, monitor = 'val_acc', verbose = 1, restore_best_weights = True)])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 128)       1280      
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 128)      512       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 128)       147584    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 128)      0         
 )                                                               
                                                                 
 batch_normalization_1 (Batc  (None, 23, 23, 128)      512       
 hNormalization)                                                 
                                                        

In [ ]:
model.evaluate(master_x_test,master_y_test,verbose=1,batch_size=1)

64000/64000 [==============================] - 204s 3ms/step - loss: 0.0039 - acc: 0.9997


[0.003865791019052267, 0.9997031092643738]

In [ ]:
import tensorflow as tf
import numpy as np
new_model = tf.keras.models.load_model('99.97%_model.h5')

In [ ]:
def predict_board(file_name):
  board = np.argmax(new_model.predict(image_preprocessor(file_name)),axis=-1)
  return np.reshape(board,(8,8))

print(predict_board("finaltest.jpg"))